In [ ]:
#Authentication Purpose
import sys
import tweepy
import re 
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler

class CustomStreamListener(tweepy.StreamListener):
    def __init__(self, maxnum, api=None):
        self.n=maxnum
        self.api=api

    def on_status(self, status):
        try:
            tweets = []
            name = status.author.screen_name
            textTwitter = status.text
            #Pre processing tweet
            Tweet = textTwitter.lower()
            Tweet = re.sub('((www\.[\s]+)|(https?://[^\s]+))','URL',Tweet)
            Tweet = re.sub('@[^\s]+','TWITTER_USER',Tweet)
            Tweet = re.sub('[\s]+', ' ', Tweet)
            Tweet = re.sub(r'#([^\s]+)', r'\1', Tweet)
            Tweet = Tweet.strip('\'"')
            #Removing emojis from tweets 
            a = ':)'
            b = ':('
            Tweet = Tweet.replace(a,'')
            Tweet = Tweet.replace(b,'') 
            
            tag = 'TWITTER_USER' 
            rt = 'rt'
            Tweet = Tweet.replace(tag,'')
            Tweet = Tweet.replace(rt,'')
            Tweet=Tweet.replace('\n', ' ').replace('\r', '')
            #Labelling tweets
            final_tweet = '|positive|, |%s| ' % (Tweet)
            #Writing into txt file
            f = open('data.txt', 'r+')
            old = f.read()
            f.write(final_tweet+'\n')
            f.close()
            self.n = self.n+1
            if self.n < 3000: 
                return True
            else:
                print ('maxnum = '+str(self.n))
                return False
        
        except Exception as e:
            print (e)

    def on_error(self, status_code):
        return True 

    def on_timeout(self):
        return True
 



Establishing stream...

maxnum = 3000


In [ ]:

class CustomStreamListenerNEG(tweepy.StreamListener):
    def __init__(self, maxnum, api=None):
        self.n = maxnum
        self.api = api

    def on_status(self, status):
        try:
            tweets = []
            name = status.author.screen_name
            textTwitter = status.text
            
            #Pre processing tweet
            Tweet = textTwitter.lower()
            Tweet = re.sub('((www\.[\s]+)|(https?://[^\s]+))','URL',Tweet)
            Tweet = re.sub('@[^\s]+','TWITTER_USER',Tweet)
            Tweet = re.sub('[\s]+', ' ', Tweet)
            Tweet = re.sub(r'#([^\s]+)', r'\1', Tweet)
            Tweet = Tweet.strip('\'"')
            #Removing emojis from tweets 
            a = ':)'
            b = ':('
            Tweet = Tweet.replace(a,'')
            Tweet = Tweet.replace(b,'') 
            
            tag = 'TWITTER_USER' 
            rt = 'rt'
            Tweet = Tweet.replace(tag,'')
            Tweet = Tweet.replace(rt,'')
            Tweet=Tweet.replace('\n', ' ').replace('\r', '')
            #Labelling tweets
            final_tweet = '|negative|, |%s| ' % (Tweet)
            # Writing into text file
            f = open('data.txt', 'r+')
            old = f.read()
            f.write(final_tweet+'\n')
            f.close()
        
            self.n = self.n+1
            if self.n < 3000: 
                return True
            else:
                print('maxnum = '+str(self.n))
                return False
        
        except Exception as e:
            print (e)

    def on_error(self, status_code):
        return True

    def on_timeout(self):
        return True

stopWords = []


In [ ]:
def replaceTwoOrMore(s):
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s) 



In [ ]:
def getStopWordList(stopWordListFileName):
    stopWords = []
    stopWords.append('TWITTER_USER')
    stopWords.append('URL')
    fp = open(stopWordListFileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stopWords.append(word)
        line = fp.readline()
    fp.close()
    return stopWords



In [ ]:
def getFeatureVector(tweet):
    
    featureVector = []
    words = tweet.split()
    for w in words:
        w = replaceTwoOrMore(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        if(w in stopWords or val is None):
            continue
        else:
            featureVector.append(w.lower())
    return featureVector



In [ ]:
#fetching data
def mainPOS():
    #Authentication
    consumer_key="nFu2HqrelkEiax0L5Lh4Sw"
    consumer_secret="6OUaIfj0ECfeJD24CVlDrcc1qqajnHBgsB7b6RPmvA"
    access_token="1193875656-G3iatRJ18tCCFTf8x06kV5B6XwdWbja4S4DVXDL"
    access_token_secret="IGOxuEQYeTREpGkL4F5LkdYNxUPLFo0zBjX3Yfdg8g"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth) 
    print("Establishing stream...\n")
    sapi = tweepy.streaming.Stream(auth, CustomStreamListener(maxnum=0))
    setTerms = [':)']
    sapi.filter(None, setTerms, languages=["en"])



In [ ]:
def mainNEG():
    consumer_key="nFu2HqrelkEiax0L5Lh4Sw"
    consumer_secret="6OUaIfj0ECfeJD24CVlDrcc1qqajnHBgsB7b6RPmvA"
    access_token="1193875656-G3iatRJ18tCCFTf8x06kV5B6XwdWbja4S4DVXDL"
    access_token_secret="IGOxuEQYeTREpGkL4F5LkdYNxUPLFo0zBjX3Yfdg8g"
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth) 
    NegTweets = tweepy.streaming.Stream(auth, CustomStreamListenerNEG(maxnum=0))
    setTermsNEG = [':(']
    NegTweets.filter(None, setTermsNEG, languages=["en"])
     
    

In [ ]:
def Positive():
    fp = open('data.txt', 'r')
    line = fp.readline()
    st = open('Sentiment_StopWords.txt', 'r')
    stopWords = getStopWordList('Sentiment_StopWords.txt')
    while line:
        featureVector=getFeatureVector(line)
        line=fp.readline()         
    


In [ ]:
if __name__ == '__main__':
    try:
        mainPOS()
        mainNEG()
        Positive()
        
    except KeyboardInterrupt:
        print("Disconnecting from Twitter... ",)
        print("Done")        
        